d-sandbox

<div style="text-align: center; line-height: 0; padding-top: 9px;">
  <img src="https://databricks.com/wp-content/uploads/2018/03/db-academy-rgb-1200px.png" alt="Databricks Learning" style="width: 600px; height: 163px">
</div>

# Logistic Regression Classifier
## Module 4 Assignment

This final assignment is broken up into 2 parts:
1. Completing this Logistic Regression Classifier notebook
  * Submitting question answers to Coursera
  * Uploading notebook to Coursera for peer reviewing
2. Answering 3 free response questions on Coursera platform

## ![Spark Logo Tiny](https://files.training.databricks.com/images/105/logo_spark_tiny.png) In this notebook you:
* Preprocess data for use in a machine learning model
* Step through creating a sklearn logistic regression model for classification
* Predict the `Call_Type_Group` for incidents in a SQL table


For each **bold** question, input its answer in Coursera.

In [0]:
%run ../Includes/Classroom-Setup

Data mounted to /mnt/davis ...

Load the `/mnt/davis/fire-calls/fire-calls-clean.parquet` data as `fireCallsClean` table.

In [0]:
%sql
USE DATABRICKS;
CREATE TABLE IF NOT EXISTS fireCallsClean
USING parquet
OPTIONS (
    path "/mnt/davis/fire-calls/fire-calls-clean.parquet"
)


Check that your data is loaded in properly.

In [0]:
%sql
SELECT * FROM fireCallsClean LIMIT 10

Call_Number,Unit_ID,Incident_Number,Call_Type,Call_Date,Watch_Date,Received_DtTm,Entry_DtTm,Dispatch_DtTm,Response_DtTm,On_Scene_DtTm,Transport_DtTm,Hospital_DtTm,Call_Final_Disposition,Available_DtTm,Address,City,Zipcode_of_Incident,Battalion,Station_Area,Box,Original_Priority,Priority,Final_Priority,ALS_Unit,Call_Type_Group,Number_of_Alarms,Unit_Type,Unit_sequence_in_call_dispatch,Fire_Prevention_District,Supervisor_District,Neighborhooods_-_Analysis_Boundaries,Location,RowID
141600888,65,14055109,Traffic Collision,06/09/2014,06/09/2014,06/09/2014 09:35:33 AM,06/09/2014 09:36:46 AM,06/09/2014 09:37:43 AM,06/09/2014 09:37:55 AM,06/09/2014 09:43:37 AM,06/09/2014 09:59:00 AM,06/09/2014 10:34:12 AM,Code 2 Transport,06/09/2014 11:14:57 AM,OAKDALE AV/TOLAND ST,San Francisco,94124,B10,09,6377,2,2,2,true,Non Life-threatening,1,MEDIC,1,10,10,Bayview Hunters Point,"(37.740961928907, -122.401555700705)",141600888-65
162743687,E01,16108733,Medical Incident,09/30/2016,09/30/2016,09/30/2016 08:05:57 PM,09/30/2016 08:07:53 PM,09/30/2016 08:08:14 PM,09/30/2016 08:08:23 PM,09/30/2016 08:10:37 PM,null,null,Code 2 Transport,09/30/2016 08:32:48 PM,1100 Block of MISSION ST,San Francisco,94103,B02,36,2318,2,2,2,true,Non Life-threatening,1,ENGINE,1,2,6,South of Market,"(37.7777124404316, -122.412736707425)",162743687-E01
102210202,75,10069623,Medical Incident,08/09/2010,08/09/2010,08/09/2010 01:28:40 PM,08/09/2010 01:30:47 PM,08/09/2010 01:31:53 PM,08/09/2010 01:32:29 PM,08/09/2010 01:52:28 PM,08/09/2010 02:19:50 PM,08/09/2010 02:30:50 PM,Code 2 Transport,08/09/2010 03:05:25 PM,100 Block of CLINTON PARK,SF,94103,B02,36,5126,1,1,2,true,null,1,MEDIC,1,2,8,Mission,"(37.7692677111289, -122.423396856968)",102210202-75
160681260,E42,16027085,Medical Incident,03/08/2016,03/08/2016,03/08/2016 10:42:26 AM,03/08/2016 10:42:57 AM,03/08/2016 10:43:20 AM,03/08/2016 10:50:11 AM,03/08/2016 10:50:11 AM,null,null,Patient Declined Transport,03/08/2016 10:58:43 AM,2400 Block of SAN BRUNO AVE,San Francisco,94134,B10,42,6362,3,3,3,true,Potentially Life-Threatening,1,ENGINE,1,10,9,Portola,"(37.7318198889718, -122.405412091734)",160681260-E42
113200298,E18,11106370,Structure Fire,11/16/2011,11/16/2011,11/16/2011 05:13:01 PM,11/16/2011 05:13:01 PM,11/16/2011 05:13:09 PM,11/16/2011 05:13:54 PM,11/16/2011 05:16:30 PM,null,null,Fire,11/16/2011 05:17:14 PM,41ST AV/NORIEGA ST,SF,94122,B08,18,7633,3,3,3,true,null,1,ENGINE,1,8,4,Sunset/Parkside,"(37.7532010612329, -122.500044865619)",113200298-E18
162584030,KM07,16101787,Medical Incident,09/14/2016,09/14/2016,09/14/2016 08:41:50 PM,09/14/2016 08:41:50 PM,09/14/2016 08:43:07 PM,09/14/2016 08:43:34 PM,09/14/2016 08:57:26 PM,null,null,Against Medical Advice,09/14/2016 09:35:01 PM,400 Block of 7TH ST,San Francisco,94103,B03,08,0231,2,2,2,false,Non Life-threatening,1,PRIVATE,1,3,6,South of Market,"(37.7746534767072, -122.405118197249)",162584030-KM07
133150239,KM09,13107112,Medical Incident,11/11/2013,11/11/2013,11/11/2013 01:41:24 PM,11/11/2013 01:44:26 PM,11/11/2013 01:44:41 PM,11/11/2013 01:45:10 PM,11/11/2013 01:51:53 PM,null,null,Gone on Arrival,11/11/2013 01:53:52 PM,16TH ST/MISSION ST,SF,94103,B02,07,5236,1,1,2,false,Non Life-threatening,1,PRIVATE,1,2,9,Mission,"(37.7650513381945, -122.419668973861)",133150239-KM09
170553524,66,17023871,Medical Incident,02/24/2017,02/24/2017,02/24/2017 07:31:51 PM,02/24/2017 07:34:55 PM,02/24/2017 07:35:42 PM,02/24/2017 07:35:51 PM,02/24/2017 07:49:49 PM,02/24/2017 08:24:58 PM,02/24/2017 08:50:07 PM,Code 2 Transport,02/24/2017 09:16:13 PM,2000 Block of 46TH AVE,San Francisco,94116,B08,23,7663,2,2,2,true,Potentially Life-Threatening,1,MEDIC,1,8,4,Sunset/Parkside,"(37.7482979256317, -122.505150416998)",170553524-66
52970075,M01,5079960,Medical Incident,10/24/2005,10/24/2005,10/24/2005 09:03:35 AM,10/24/2005 09:05:37 AM,10/24/2005 09:05:55 AM,10/24/2005 09:07:09 AM,10/24/2005 09:12:02 AM,null,null,Cancelled,10/24/2005 09:12:42 AM,400 Block of GOLDEN GATE AVE,SF,94102,B02,03,1644,3,3,3,true,null,

By the end of this assignment, we would like to train a logistic regression model to predict 2 of the most common `Call_Type_Group` given information from the rest of the table.

Write a query to see what the different `Call_Type_Group` values are and their respective counts.

### Question 1

**How many calls of `Call_Type_Group` "Fire"?**

In [0]:
%sql
SELECT
  Call_Type_Group
  , COUNT(*)
FROM fireCallsClean
GROUP BY
  Call_Type_Group

Call_Type_Group,count(1)
Alarm,32566
null,246459
Potentially Life-Threatening,78030
Non Life-threatening,56168
Fire,4196


Let's drop all the rows where `Call_Type_Group = null`. Since we don't have a lot of `Call_Type_Group` with the value `Alarm` and `Fire`, we will also drop these calls from the table. Call this new temporary view `fireCallsGroupCleaned`.

In [0]:
%sql
CREATE OR REPLACE VIEW fireCallsGroupCleaned
AS (
  SELECT *
  FROM fireCallsClean
  WHERE Call_Type_Group IS NOT NULL AND Call_Type_Group <> "Alarm" AND Call_Type_Group <> "Fire"
)


Check that every entry in `fireCallsGroupCleaned`  has a `Call_Type_Group` of either `Potentially Life-Threatening` or `Non Life-threatening`.

In [0]:
%sql
SELECT
  Call_Type_Group
FROM fireCallsGroupCleaned
GROUP BY
  Call_Type_Group

Call_Type_Group
Potentially Life-Threatening
Non Life-threatening


### Question 2

**How many rows are in `fireCallsGroupCleaned`?**

In [0]:
%sql

select count(*) from fireCallsGroupCleaned

count(1)
134198


We probably don't need all the columns of `fireCallsGroupCleaned` to make our prediction. Select the following columns from `fireCallsGroupCleaned` and create a view called `fireCallsDF` so we can access this table in Python:

* "Call_Type"
* "Fire_Prevention_District"
* "Neighborhooods_-\_Analysis_Boundaries" 
* "Number_of_Alarms"
* "Original_Priority" 
* "Unit_Type" 
* "Battalion"
* "Call_Type_Group"

In [0]:
%sql
CREATE OR REPLACE VIEW fireCallsDF
AS (
  SELECT 
    Call_Type
    , Fire_Prevention_District
    , `Neighborhooods_-_Analysis_Boundaries`
    , Number_of_Alarms
    , Original_Priority
    , Unit_Type
    , Battalion
    , Call_Type_Group
  FROM fireCallsGroupCleaned
)

Fill in the string SQL statement to load the `fireCallsDF` table you just created into python.

In [0]:
%python
# TODO
df = sql("""SELECT *
            FROM fireCallsDF""")
display(df)

Call_Type,Fire_Prevention_District,Neighborhooods_-_Analysis_Boundaries,Number_of_Alarms,Original_Priority,Unit_Type,Battalion,Call_Type_Group
Traffic Collision,10,Bayview Hunters Point,1,2,MEDIC,B10,Non Life-threatening
Medical Incident,2,South of Market,1,2,ENGINE,B02,Non Life-threatening
Medical Incident,10,Portola,1,3,ENGINE,B10,Potentially Life-Threatening
Medical Incident,3,South of Market,1,2,PRIVATE,B03,Non Life-threatening
Medical Incident,2,Mission,1,1,PRIVATE,B02,Non Life-threatening
Medical Incident,8,Sunset/Parkside,1,2,MEDIC,B08,Potentially Life-Threatening
Medical Incident,6,Castro/Upper Market,1,2,ENGINE,B05,Potentially Life-Threatening
Medical Incident,4,Nob Hill,1,2,PRIVATE,B04,Non Life-threatening
Medical Incident,9,Excelsior,1,3,ENGINE,B09,Potentially Life-Threatening
Medical Incident,3,South of Market,1,3,ENGINE,B03,Potentially Life-Threatening


## Creating a Logistic Regression Model in Sklearn

First we will convert the Spark DataFrame to pandas so we can use sklearn to preprocess the data into numbers so that it is compatible with the logistic regression algorithm with a [LabelEncoder](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.LabelEncoder.html). 

Then we'll perform a train test split on our pandas DataFrame. Remember that the column we are trying to predict is the `Call_Type_Group`.

In [0]:
%python
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

pdDF = df.toPandas()
le = LabelEncoder()
numerical_pdDF = pdDF.apply(le.fit_transform)

X = numerical_pdDF.drop("Call_Type_Group", axis=1)
y = numerical_pdDF["Call_Type_Group"].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

Look at our training data `X_train` which should only have numerical values now.

In [0]:
%python
display(X_train)

Call_Type,Fire_Prevention_District,Neighborhooods_-_Analysis_Boundaries,Number_of_Alarms,Original_Priority,Unit_Type,Battalion
0,7,10,0,1,2,6
0,2,9,0,1,5,1
0,2,29,0,2,2,9
0,3,5,0,1,4,0
0,1,1,0,1,2,5
0,4,36,0,3,5,3
2,10,37,0,2,2,2
0,5,8,0,1,2,4
0,2,36,0,1,2,2
0,2,34,0,1,4,1


We'll create a pipeline with 2 steps. 

0. [One Hot Encoding](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html#sklearn.preprocessing.OneHotEncoder): Converts our  features into vectorized features by creating a dummy column for each value in that category. 

0. [Logistic Regression model](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html): Although the name includes "regression", it is used for classification by predicting the probability that the `Call Type Group` is one label and not the other.

In [0]:
%python
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline

ohe = ("ohe", OneHotEncoder(handle_unknown="ignore"))
lr = ("lr", LogisticRegression())

pipeline = Pipeline(steps = [ohe, lr]).fit(X_train, y_train)
y_pred = pipeline.predict(X_test)

/databricks/python/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
 FutureWarning)

Run the following cell to see how well our model performed on test data (data that wasn't used to train the model)!

In [0]:
%python
from sklearn.metrics import accuracy_score
print(f"Accuracy of model: {accuracy_score(y_pred, y_test)}")

Accuracy of model: 0.8176974664679583

### Question 3

**What is the accuracy of our model on test data? Round to the nearest percent.**

Save pipeline (with both stages) to disk.

In [0]:
%python

import mlflow
from mlflow.sklearn import save_model

model_path = "/dbfs/" + username + "/Call_Type_Group_lr"
dbutils.fs.rm(username + "/Call_Type_Group_lr", recurse=True)
save_model(pipeline, model_path)

## UDF

Now that we have created and trained a machine learning pipeline, we will use MLflow to register the `.predict` function of the sklearn pipeline as a UDF which we can use later to apply in parallel. Now we can refer to this with the name `predictUDF` in SQL.

In [0]:
%python
import mlflow
from mlflow.pyfunc import spark_udf

predict = spark_udf(spark, model_path, result_type="int")
spark.udf.register("predictUDF", predict)

Out[8]: <function mlflow.pyfunc.spark_udf.<locals>.predict(*args)>

Create a view called `testTable` of our test data `X_test` so that we can see this table in SQL.

In [0]:
%python
spark_df = spark.createDataFrame(X_test)
spark_df.createOrReplaceTempView("testTable")

Create a table called `predictions` using the `predictUDF` function we registered beforehand. Apply the `predictUDF` to every row of `testTable` in parallel so that each row of `testTable` has a `Call_Type_Group` prediction.

In [0]:
%sql
USE DATABRICKS;
DROP TABLE IF EXISTS predictions;

CREATE TEMPORARY VIEW predictions 
AS (
  SELECT
    CAST(predictUDF(Call_Type
                    , Fire_Prevention_District
                    , `Neighborhooods_-_Analysis_Boundaries`
                    , Number_of_Alarms
                    , Original_Priority
                    , Unit_Type
                    , BattalioN) AS double) AS predictions
    , *
   FROM testTable
);


Now take a look at the table and see what your model predicted for each call entry!

In [0]:
%sql
SELECT * FROM predictions LIMIT 10

predictions,Call_Type,Fire_Prevention_District,Neighborhooods_-_Analysis_Boundaries,Number_of_Alarms,Original_Priority,Unit_Type,Battalion
1.0,0,3,34,0,2,2,2
0.0,2,1,0,0,1,2,9
0.0,0,9,25,0,1,4,8
0.0,0,3,34,0,1,5,2
1.0,0,2,18,0,2,4,1
0.0,0,3,34,0,1,4,2
0.0,0,8,35,0,1,4,7
1.0,0,2,18,0,2,2,1
0.0,0,2,34,0,1,4,1
0.0,0,4,41,0,1,4,3


### Question 4: 

**What 2 values are in the `prediction` column?**

1 et 0

Congrats on finishing your last assignment notebook! 


Now you will have to upload this notebook to Coursera for peer reviewing.
1. Make sure that all your code will run without errors
  * Check this by clicking the "Clear State & Run All" dropdown option at the top of your notebook
  * ![](http://files.training.databricks.com/images/eLearning/ucdavis/clearstaterunall.png)
2. Click on the "Workspace" icon on the side bar
3. Next to the notebook you're working in right now, click on the dropdown arrow
4. In the dropdown, click on "Export" then "HTML"
  * ![](http://files.training.databricks.com/images/eLearning/ucdavis/export.png)
5. On the Coursera platform, upload this HTML file to Week 4's Peer Review Assignment

Go back onto the Coursera platform for the free response portion of this assignment and for instructions on how to review your peer's work.

-sandbox
&copy; 2020 Databricks, Inc. All rights reserved.<br/>
Apache, Apache Spark, Spark and the Spark logo are trademarks of the <a href="http://www.apache.org/">Apache Software Foundation</a>.<br/>
<br/>
<a href="https://databricks.com/privacy-policy">Privacy Policy</a> | <a href="https://databricks.com/terms-of-use">Terms of Use</a> | <a href="http://help.databricks.com/">Support</a>